In [ ]:
'''
匯入套件
'''
# 操作 browser 的 API
# 用來呼叫瀏覽器
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的，用來下指令
import os

'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options ) #跑了之後會開一個無痕視窗，會有寫「Chrome 目前受到自動測試軟體控制」

# driver.set_window_size(1200, 960) #視窗大小設定 (寬，高)
# driver.maximize_window() #視窗最大化
# driver.minimize_window() #視窗最小化

# 放置爬取的資料
listData = []

'''
以 function 名稱，作為爬蟲流程
'''

# 走訪頁面
def visit():
    driver.get('https://www.youtube.com/');  #get: 把網址寫在瀏覽器

# 輸入關鍵字
def search():
    # 輸入名稱
    txtInput = driver.find_element(By.CSS_SELECTOR, "input#search")  # find_element: 找元素，此處為找搜尋列
    # 單一元素:find_element  多元素: find_elements  
    # 可在youtube的搜尋列按右鍵選檢查，可看到一些id之類的東西
    # CSS: id --> #    Class --> .
    # input#search: 告訴電腦我input要寫id為search
    # 回去查「CSS選擇器」、把它搞清楚
    txtInput.send_keys("張學友")  # send_keys: 幫你打字，也可用來輸帳號密碼、寫文章之類的
    
    #按下送出
    btnInput = driver.find_element(By.CSS_SELECTOR, "button#search-icon-legacy")
    # 在youtube上按搜尋的按鈕右鍵、檢查，然後在開發人員工具的elements裡隨便找地方按ctrl+f可以用CSS來filter
    
    btnInput.click()
    
# 篩選 (選項)
def filterFunc():
    try:
        # 等待篩選元素出現
        WebDriverWait(driver, 10).until(  #要電腦幫你等十秒，直到filters這個元素在網頁上產生 (這個元素出現就表示網站已經跑完了)
            EC.presence_of_element_located( 
                (By.CSS_SELECTOR, "yt-formatted-string#text.style-scope.ytd-toggle-button-renderer.style-text")   #tuple
            )
        )

        #按下篩選元素，使項目浮現
        driver.find_element(
            By.CSS_SELECTOR, 
            "yt-formatted-string#text.style-scope.ytd-toggle-button-renderer.style-text").click()  #找filter那個按鈕，然後按下去

        # 等待一下
        sleep(2)

        # 按下選擇的項目
        driver.find_elements(
            By.CSS_SELECTOR, 
            "yt-formatted-string.style-scope.ytd-search-filter-renderer")[10].click() #在youtube找短片，實際上是27個裡面的第11個，但python是從0開始、所以在python裡是第10個
        
    except TimeoutException:  
        print("等待逾時，即將關閉瀏覽器…")
        sleep(3)
        driver.quit()
        
# 滾動頁面
def scroll():
    # 瀏覽器內部的高度，當你一直滾、快滾到底的時候會更新
    innerHeightOfWindow = 0
    
    # 當前捲動的量(高度)
    totalOffset = 0
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while totalOffset <= innerHeightOfWindow:
        # 每次移動高度
        totalOffset += 300;
        
        # 捲動的 js code  #下面這段是java script的程式
        js_scroll = '''(
            function (){{
                window.scrollTo({{
                    top:{}, 
                    behavior: 'smooth' 
                }});
            }})();'''.format(totalOffset)
        # 每次會滾到top後面{}的值，並透過.format(totalOffset)把值放到{}裡面
        # behavior: "smooth" 表示平滑的滑動，而不是一次跳到那個位置
        
        # 執行 js code
        driver.execute_script(js_scroll)
        
        # 強制等待
        sleep(1)
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeightOfWindow = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        );
        
        # 強制等待
        sleep(1)
        
        # 印出捲動距離
        print("innerHeightOfWindow: {}, totalOffset: {}".format(innerHeightOfWindow, totalOffset))
        
        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if totalOffset >= 1800:
            break

# 分析頁面元素資訊
def parse():
    # 取得主要元素的集合
    ytd_video_renderers = driver.find_elements(
        By.CSS_SELECTOR, 
        'ytd-video-renderer.style-scope.ytd-item-section-renderer')
    
    # 逐一檢視元素
    for element in ytd_video_renderers:
        # 印出分隔文字
        print("=" * 30)
        
        # 取得圖片連結
        img = element.find_element(
            By.CSS_SELECTOR, 
            "ytd-thumbnail.style-scope.ytd-video-renderer img#img")
        imgSrc = img.get_attribute('src')
        print(imgSrc)
        
        # 取得資料名稱
        a = element.find_element(By.CSS_SELECTOR, "a#video-title")
        aTitle = a.text
        print(aTitle)
        
        # 取得 YouTube 連結
        aLink = a.get_attribute('href')
        print(aLink)
        
        # 放資料到 list 中
        listData.append({
            "imgSrc": imgSrc,
            "aTitle": aTitle,
            "aLink": aLink
        })

# 將 list 存成 json
def saveJson():
    fp = open("youtube.json", "w", encoding='utf-8')  # 開一個新檔案叫"youtube.json"，w for write、寫入資料
    fp.write( json.dumps(listData, ensure_ascii=False) )  #json.dumps: 從變數型態轉成文字型態 #ensure_ascii: 免得中文變成一堆%20之類的東西
    fp.close()
    
# 關閉瀏覽器
def close():
    driver.quit()  # or driver.close()

# ??????
def download():
    # 開啟 json 檔案
    fp = open("youtube.json", "r", encoding='utf-8')  # r for read
    
    #取得 json 字串
    strJson = fp.read()  #把檔案裡的內容存到變數裡去
    
    # 關閉檔案
    fp.close()  #有開就要有關
    
    # 將 json 轉成 list (裡面是 dict 集合)
    listResult = json.loads(strJson)  #loads: 跟dumps相反，把文字變成變數 --> list
    
    # ??????
    for index, obj in enumerate(listResult):
        if index <= 2:  #測試時為免花太多時間，只抓3首歌
            os.system("youtube-dl.exe -f mp4 -i {} -o {}".format(obj["aLink"], "%(id)s.%(ext)s"))  # os.system: 類似你自己開終端機、打程式碼
                                    # -f: 指定副檔名為mp4  # -i: 後面貼要下載的網址 # -o:指定儲存名稱為「id.副檔名(mp4)」  


# 主程式
if __name__ == '__main__':
    visit()
    search()
    filterFunc()
    scroll()
    parse()
    saveJson()
    close()
    download()